In [ ]:
import pandas as pd
from os import mkdir

# Tratamento da Base de Dados

Antes de começar, faça o download dos dados originais, descompacte-os e os coloque numa pasta chamada "dados-originais" no mesmo diretório desse arquivo main.
https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads

## Mapeamento e tradução

Toda a base de dados usa os dados em um formato de números para representar possíveis respostas, então é preciso fazer um mapeamento dos dados desses valores numéricos para o que eles realmente significam.

As colunas estão descritas em forma de código do questionário usado, exemplos: 'A001A', 'B00113', 'F001'. Então aqui traduziremos as colunas para que fiquem de uma forma mais legível. Para isso temos a função `traduzir_colunas()`

### Mapeamento dos Dados e Dicionário das Colunas

In [ ]:
mapeamento_situacao_domicilio = {
    1: 'Urbana',
	2: 'Rural'
}

mapeamento_condicao_domicilio = {
    1: 'Pessoa responsável pelo domicílio',
    2: 'Cônjuge ou companheiro(a) de sexo diferente',
    3: 'Cônjuge ou companheiro(a) do mesmo sexo',
    4: 'Filho(a) do responsável e do cônjuge',
    5: 'Filho(a) somente do responsável',
    6: 'Filho(a) somente do cônjuge',
    7: 'Genro ou nora',
    8: 'Pai, mãe, padrasto ou madrasta',
    9: 'Sogro(a)',
    10: 'Neto(a)',
    11: 'Bisneto(a)',
    12: 'Irmão ou irmã',
    13: 'Avô ou avó',
    14: 'Outro parente',
    15: 'Agregado(a) - Não parente que não compartilha despesas',
    16: 'Convivente - Não parente que compartilha despesas',
    17: 'Pensionista',
    18: 'Empregado(a) doméstico(a)',
    19: 'Parente do(a) empregado(a) doméstico(a)'
}

mapeamento_sexo = {
    1: 'Homem',
	2: 'Mulher'
}

mapeamento_cor_raca = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indígena',
    9: 'Ignorado'
}

mapeamento_escolaridade = {
    1: 'Sem instrução',
    2: 'Fundamental incompleto',
    3: 'Fundamental completa',
    4: 'Médio incompleto',
    5: 'Médio completo',
    6: 'Superior incompleto',
    7: 'Superior completo',
    8: 'Pós-graduação, mestrado ou doutorado'
}

mapeamento_frequenta_escola = {
    1: 'Sim',
    2: 'Não'
}

mapeamento_escola_publica_privada = {
    1: 'Pública',
    2: 'Privada'
}

mapeamento_aulas_presenciais = {
    1: 'Sim, normalmente',
    2: 'Sim, mas apenas parcialmente',
    3: 'Não, e meu normalmente é presencial/semipresencial',
    4: 'Não, meu curso é online'
}

mapeamento_sintomas = {
    1: 'Sim',
    2: 'Não',
    3: 'Não sabe',
    9: 'Ignorado'
}

mapeamento_teste_covid = {
    1: 'Sim',
    2: 'Não',
    9: 'Ignorado'
}

mapeamento_resultado_teste = {
    1: 'Positivo',
    2: 'Negativo',
    3: 'Inconclusivo',
    4: 'Ainda não recebeu o resultado',
    9: 'Ignorado'
}

mapeamento_tipo_trabalho = {
    1: 'Trabalhador doméstico (empregado doméstico, cuidados, babá)',
    2: 'Militar do exército, marinha ou aeronáutica',
    3: 'Policial militar ou bombeiro militar',
    4: 'Empregado do setor privado',
    5: 'Empregado do setor público (inclusive empresas de economia mista)',
    6: 'Empregador',
    7: 'Conta própria',
    8: 'Trabalhador familiar não remunerado em ajuda a membro do domicílio ou parente',
    9: 'Estava fora do mercado de trabalho (fazia apenas afazeres domésticos, cuidados de pessoas ou produção para próprio consumo)'
}

mapeamento_area_trabalho = {
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal'
}

mapeamento_carteira_trabalho = {
    1: 'Sim, tem carteira de trabalho assinada',
    2: 'Sim, é servidor público estatutário',
    3: 'Não'
}

mapeamento_tipo_cargo = {
    1: 'Empregado doméstico, diarista, cozinheiro (em domicílios particulares)',
    2: 'Faxineiro, auxiliar de limpeza etc. (em empresa pública ou privada)',
    3: 'Auxiliar de escritório, escriturário',
    4: 'Secretária, recepcionista',
    5: 'Operador de Telemarketing',
    6: 'Comerciante (dono do bar, da loja etc.)',
    7: 'Balconista, vendedor de loja',
    8: 'Vendedor a domicílio, representante de vendas, vendedor de catálogo (Avon, Natura etc.)',
    9: 'Vendedor ambulante (feirante, camelô, comerciante de rua, quiosque)',
    10: 'Cozinheiro e garçom (de restaurantes, empresas)',
    11: 'Padeiro, açougueiro e doceiro',
    12: 'Agricultor, criador de animais, pescador, silvicultor e jardineiro',
    13: 'Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)',
    14: 'Motorista (de aplicativo, de taxi, de van, de mototáxi, de ônibus)',
    15: 'Motorista de caminhão (caminhoneiro)',
    16: 'Motoboy',
    17: 'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Eats, IFood, Rappy etc.)',
    18: 'Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro',
    19: 'Mecânico de veículos, máquinas industriais etc.',
    20: 'Artesão, costureiro e sapateiro',
    21: 'Cabeleireiro, manicure e afins',
    22: 'Operador de máquinas, montador na indústria',
    23: 'Auxiliar de produção, de carga e descarga',
    24: 'Professor da educação infantil, de ensino fundamental, médio ou superior',
    25: 'Pedagogo, professor de idiomas, música, arte e reforço escolar',
    26: 'Médico, enfermeiro, profissionais de saúde de nível superior',
    27: 'Técnico, profissional da saúde de nível médio',
    28: 'Cuidador de crianças, doentes ou idosos',
    29: 'Segurança, vigilante, outro trabalhador dos serviços de proteção',
    30: 'Policial civil',
    31: 'Porteiro, zelador',
    32: 'Artista, religioso (padre, pastor etc.)',
    33: 'Diretor, gerente, cargo político ou comissionado',
    34: 'Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)',
    35: 'Outro técnico ou profissional de nível médio',
    36: 'Outros'
}

mapeamento_atividade_empresa = {
    1: 'Agricultura, pecuária, produção florestal e pesca',
    2: 'Extração de petróleo, carvão mineral, minerais metálicos, pedra, areia, sal etc.',
    3: 'Indústria da transformação (inclusive confecção e fabricação caseira)',
    4: 'Fornecimento de eletricidade e gás, água, esgoto e coleta de lixo',
    5: 'Construção',
    6: 'Comércio no atacado e varejo',
    7: 'Reparação de veículos automotores e motocicletas',
    8: 'Transporte de passageiros',
    9: 'Transporte de mercadorias',
    10: 'Armazenamento, correios e serviços de entregas',
    11: 'Hospedagem (hotéis, pousadas etc.)',
    12: 'Serviço de alimentação (bares, restaurantes, ambulantes de alimentação)',
    13: 'Informação e comunicação (jornais, rádio e televisão, telecomunicações e informática)',
    14: 'Bancos, atividades financeiras e de seguros',
    15: 'Atividades imobiliárias',
    16: 'Escritórios de advocacia, engenharia, publicidade e veterinária (Atividades profissionais, científicas e técnicas)',
    17: 'Atividades de locação de mão de obra, segurança, limpeza, paisagismo e teleatendimento',
    18: 'Administração pública (governo federal, estadual e municipal)',
    19: 'Educação',
    20: 'Saúde humana e assistência social',
    21: 'Organizações religiosas, sindicatos e associações',
    22: 'Atividade artísticas, esportivas e de recreação',
    23: 'Cabeleireiros, tratamento de beleza e serviços pessoais',
    24: 'Serviço doméstico remunerado (será imputado da posição na ocupação)',
    25: 'Outro'
}

mapeamento_faixa_rendimento = {
    0: '0 - 100',
    1: '101 - 300',
    2: '301 - 600',
    3: '601 - 800',
    4: '801 - 1.600',
    5: '1.601 - 3.000',
    6: '3.001 - 10.000',
    7: '10.001 - 50.000',
    8: '50.001 - 100.000',
    9: 'Mais de 100.000'
}

mapeamento_tipo_domicilio = {
    1: 'Próprio - já pago',
    2: 'Próprio - ainda pagando',
    3: 'Alugado',
    4: 'Cedido por empregador',
    5: 'Cedido por familiar',
    6: 'Cedido de outra forma',
    7: 'Outra condição'
}

dicionario_colunas = {
    'A002': 'Idade',
    'A003': 'Sexo',
    'A004': 'Cor',
    'A005': 'Escolaridade',
    'A006': 'Frequenta_escola',
    'A006A': 'Escola_eh',
    'A006B': 'Aulas_presenciais',
    'B0011': 'Febre',
    'B0012': 'Tosse',
    'B0013': 'Dor_garganta',
    'B0014': 'Dificuldade_respirar',
    'B0015': 'Dor_cabeca',
    'B0016': 'Dor_peito',
    'B0017': 'Nausea',
    'B0018': 'Nariz_entupido',
    'B0019': 'Fadiga',
    'B00110': 'Dor_olhos',
    'B00111': 'Perda_cheiro',
    'B00112': 'Dor_muscular',
    'B00113': 'Diarreia',
    'B008': 'Teste_coronavírus',
    'B009A': 'Exame SWAB',
    'B009B': 'Resultado_SWAB',
    'B009C': 'Exame_sangue_dedo',
    'B009D': 'Resultado_sangue_dedo',
    'B009E': 'Exame_sangue_braco',
    'B009F': 'Resultado_sangue_braco',
    'C007': 'Trabalho',
    'C007A': 'Area_trabalho',
    'C01011': 'Faixa_renda'
}

### Funções

In [ ]:
def mapear(df, coluna, dicionario_map):
    if coluna in df.columns:
        df[coluna] = df[coluna].map(dicionario_map)

def traduzir_dados(df: pd.DataFrame):
    # Aplicar as traduções para cada coluna
    mapear(df, 'V1022', mapeamento_situacao_domicilio)
    mapear(df, 'A001A', mapeamento_condicao_domicilio)
    mapear(df, 'A003', mapeamento_sexo)
    mapear(df, 'A004', mapeamento_cor_raca)
    mapear(df, 'A005', mapeamento_escolaridade)
    mapear(df, 'A006', mapeamento_frequenta_escola)
    mapear(df, 'A006A', mapeamento_escola_publica_privada)
    mapear(df, 'A006B', mapeamento_aulas_presenciais)
    
    # Traduzir sintomas e testes COVID-19
    colunas_sintomas = [
        'B0011', 'B0012', 'B0013', 'B0014', 'B0015', 'B0016', 'B0017',
        'B0018', 'B0019', 'B00110', 'B00111', 'B00112', 'B00113' 
    ]
    
    for coluna in colunas_sintomas:
        mapear(df, coluna, mapeamento_sintomas)

    colunas_testes = [
        'B008', 'B009A', 'B009C', 'B009E'
    ]
    for coluna in colunas_testes:
        mapear(df, coluna, mapeamento_teste_covid)

    colunas_resultados = [
        'B009B', 'B009D', 'B009F'
    ]
    
    for coluna in colunas_resultados:
        mapear(df, coluna, mapeamento_resultado_teste)

    # Traduzir informações de trabalho
    mapear(df, 'C007', mapeamento_tipo_trabalho)
    mapear(df, 'C007A', mapeamento_area_trabalho)
    mapear(df, 'C007B', mapeamento_carteira_trabalho)
    mapear(df, 'C007C', mapeamento_tipo_cargo)
    mapear(df, 'C007D', mapeamento_atividade_empresa)

    # Traduzir rendimento e domicílio
    mapear(df, 'C01011', mapeamento_faixa_rendimento)
    mapear(df, 'F001', mapeamento_tipo_domicilio)

def traduzir_colunas(df):
    for coluna in df.columns:
        if coluna in dicionario_colunas:
            df.rename(columns={coluna: dicionario_colunas[coluna]}, inplace=True)

# Tratamento dos dados

In [ ]:
def filtrar_paraiba(df):
    df_paraiba = df.copy()
    df_paraiba = df[df['UF'] == 25]
    return df_paraiba

def filtrar_colunas_desejadas(df: pd.DataFrame):
    colunas_desejadas = list(dicionario_colunas.keys())
    
    colunas_existentes = [coluna for coluna in colunas_desejadas if coluna in df.columns]
    
    return df[colunas_existentes]

def tratamento_dados(df):
    # df = filtrar_paraiba(df)
    df = filtrar_colunas_desejadas(df)
    traduzir_dados(df)
    traduzir_colunas(df)
    return df

In [ ]:
try:
    mkdir('dados-tratados')
    print('Pasta "dados-tratados" criado com sucesso.')
except FileExistsError:
    print('Pasta "dados-tratados" já existe.')

for i in range(5, 12):
    arquivo = f'dados-originais/PNAD_COVID_{i:0>2}2020.csv'
    
    try:
        df = pd.read_csv(arquivo)
    except FileNotFoundError:
        print(f'Falha ao ler o arquivo "{arquivo}".')
        continue

    df = tratamento_dados(df)

    df.to_csv(f'dados-tratados/pc-{i:0>2}.csv',sep = ";")
    
    display(df)